In [2]:
import numpy as np

# Initialize the lattice
def initialize_lattice(L):
    """Initialize an LxL lattice with random spin configuration (+1 or -1)."""
    return np.random.choice([-1, 1], size=(L, L))

# Compute the energy change for a proposed spin flip with external magnetic field
def delta_energy(lattice, i, j, H):
    """Calculate the change in energy when flipping the spin at (i, j), including external field H."""
    L = lattice.shape[0]
    spin = lattice[i, j]
    neighbors_sum = (
        lattice[(i + 1) % L, j] +
        lattice[(i - 1) % L, j] +
        lattice[i, (j + 1) % L] +
        lattice[i, (j - 1) % L]
    )
    dE = 2 * spin * (neighbors_sum + H)  # Include external magnetic field H
    return dE

# Metropolis-Hastings step
def metropolis_step(lattice, beta, H):
    """Perform one Metropolis-Hastings step for the entire lattice."""
    L = lattice.shape[0]
    for _ in range(L * L):  # Attempt L^2 spin flips per step
        i = np.random.randint(L)
        j = np.random.randint(L)
        
        dE = delta_energy(lattice, i, j, H)
        
        # Metropolis criterion
        if dE <= 0 or np.random.rand() < np.exp(-beta * dE):
            lattice[i, j] *= -1  # Flip the spin

# Run the simulation and calculate equilibrium magnetization
def run_simulation(L, beta, H, steps, equilibration_steps=100):
    """Run the Metropolis-Hastings simulation and calculate magnetization."""
    lattice = initialize_lattice(L)
    
    # Equilibrate the system
    for _ in range(equilibration_steps):
        metropolis_step(lattice, beta, H)
    
    # Calculate magnetization after equilibration
    magnetizations = []
    for step in range(steps):
        metropolis_step(lattice, beta, H)
        magnetization = np.sum(lattice) / (L * L)  # Magnetization per site
        magnetizations.append(magnetization)
    
    # Return average magnetization as equilibrium value
    # also the standard deviation of the magnetization 
    # to show the convergence of the MC simulation
    return np.mean(magnetizations), np.std(magnetizations)



In [4]:
# Parameters
L = 20  # Lattice size (LxL)
steps = 1000  # Number of Monte Carlo steps for measurement
equilibration_steps = 500  # Steps to reach equilibrium

# Temperature and external field range
temperatures = np.linspace(1.5, 5.0, 20)  # Inverse temperature range (1/kT)
H_values = [0, 0.1, 0.5]  # Different external magnetic field strengths

# Run the simulation for different temperatures and fields
mag_avg_all = {}
mag_std_all = {}
for H in H_values:
    mag_avg_tmp = []
    mag_std_tmp = []
    for T in temperatures:
        print(T)
        beta = 1 / T
        mag_avg, mag_std = run_simulation(L, beta, H, steps, equilibration_steps)
        mag_avg_tmp.append(mag_avg)
        mag_std_tmp.append(mag_std)

    mag_avg_all[H] = mag_avg_tmp
    mag_std_all[H] = mag_std_tmp

1.5
1.6842105263157894
1.868421052631579
2.052631578947368
2.236842105263158
2.4210526315789473
2.6052631578947367
2.7894736842105265
2.973684210526316
3.1578947368421053


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Plot results
# plt.figure(figsize=(10, 6))
plt.figure()

for iH in range(len(H_values)):
    H_tmp = H_values[iH]
    plt.errorbar(
        temperatures, 
        np.abs(mag_avg_all[H_tmp]), 
        np.abs(mag_std_all[H_tmp]), 
        fmt='o-', 
        label=f'H = {H_tmp}'
    )

# theoretical critcal temperature at H=0: Tc = 2.269*J
plt.axvline(x = 2.269, color = 'b', ls='--')
    
plt.xlabel('Temperature (T)')
plt.ylabel('Equilibrium Magnetization')
plt.title('Equilibrium Magnetization vs Temperature')
plt.legend()
plt.grid(True)
# plt.show()

plt.savefig("Ising_2D.pdf", dpi=150)